In [50]:
pip install findspark

In [51]:
# Import findspark and initialize.
import findspark
findspark.init()

In [52]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [53]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add the file to Spark using the URL
spark.sparkContext.addFile(url)

# Read the CSV file into a DataFrame
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the DataFrame to verify
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [54]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

# Verify the temporary view by running a simple SQL query
spark.sql("SELECT * FROM home_sales LIMIT 5").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [55]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT
    YEAR(date) AS Year,
    Concat('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS Average_Price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    YEAR(date)
ORDER BY
    year
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()


+----+-------------+
|Year|Average_Price|
+----+-------------+
|2019|  $300,263.70|
|2020|  $298,353.78|
|2021|  $301,819.44|
|2022|  $296,363.88|
+----+-------------+



In [56]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = """
SELECT
    date_built AS Year_Built,
    Concat('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS Average_Price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|  $292,859.62|
|      2011|  $291,117.47|
|      2012|  $293,683.19|
|      2013|  $295,962.27|
|      2014|  $290,852.27|
|      2015|  $288,770.30|
|      2016|  $290,555.07|
|      2017|  $292,676.79|
+----------+-------------+



In [57]:
# SQL query to calculate the average price for 3-bedroom, 3-bathroom, two-floor homes
# greater than or equal to 2,000 square feet, for each year built

query = """
SELECT
    date_built AS Year_built,
    CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS Average_Price
FROM
    home_sales
WHERE
    bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()





+----------+-------------+
|Year_built|Average_Price|
+----------+-------------+
|      2010|  $285,010.22|
|      2011|  $276,553.81|
|      2012|  $307,539.97|
|      2013|  $303,676.79|
|      2014|  $298,264.72|
|      2015|  $297,609.97|
|      2016|  $293,965.10|
|      2017|  $280,317.58|
+----------+-------------+



In [58]:
# Start the timer
start_time = time.time()

# SQL query to calculate the average price per view rating
query = """
SELECT
    view AS View_rating,
    CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS Average_Price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view_rating DESC
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

# Calculate and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))


+-----------+-------------+
|View_rating|Average_Price|
+-----------+-------------+
|        100|$1,026,669.50|
|         99|$1,061,201.42|
|         98|$1,053,739.33|
|         97|$1,129,040.15|
|         96|$1,017,815.92|
|         95|$1,054,325.60|
|         94|$1,033,536.20|
|         93|$1,026,006.06|
|         92|  $970,402.55|
|         91|$1,137,372.73|
|         90|$1,062,654.16|
|         89|$1,107,839.15|
|         88|$1,031,719.35|
|         87|$1,072,285.20|
|         86|$1,070,444.25|
|         85|$1,056,336.74|
|         84|$1,117,233.13|
|         83|$1,033,965.93|
|         82|$1,063,498.00|
|         81|$1,053,472.79|
+-----------+-------------+
only showing top 20 rows

--- 0.6411511898040771 seconds ---


In [59]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [60]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [61]:
# Verify that the table is cached with a simple Query
spark.sql("SELECT * FROM home_sales LIMIT 5").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [62]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# SQL query to calculate the average price per view rating
query = """
SELECT
    view AS View_rating,
    CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS Average_Price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view_rating DESC
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+-----------+-------------+
|View_rating|Average_Price|
+-----------+-------------+
|        100|$1,026,669.50|
|         99|$1,061,201.42|
|         98|$1,053,739.33|
|         97|$1,129,040.15|
|         96|$1,017,815.92|
|         95|$1,054,325.60|
|         94|$1,033,536.20|
|         93|$1,026,006.06|
|         92|  $970,402.55|
|         91|$1,137,372.73|
|         90|$1,062,654.16|
|         89|$1,107,839.15|
|         88|$1,031,719.35|
|         87|$1,072,285.20|
|         86|$1,070,444.25|
|         85|$1,056,336.74|
|         84|$1,117,233.13|
|         83|$1,033,965.93|
|         82|$1,063,498.00|
|         81|$1,053,472.79|
+-----------+-------------+
only showing top 20 rows

--- 0.454836368560791 seconds ---


In [63]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Write the DataFrame to Parquet format, partitioned by the "date_built" field
output_path = "/path/to/output/home_sales_partitioned"

df.write.partitionBy("date_built").mode("overwrite").parquet(output_path)


In [64]:
# 11. Read the formatted parquet data.
# Path to the partitioned Parquet data
input_path = "/path/to/output/home_sales_partitioned"

# Read the Parquet data into a DataFrame
parquet_df = spark.read.parquet(input_path)

# Show the DataFrame to verify the data
parquet_df.show(5)


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [65]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

# Verify the temporary table by running a query
spark.sql("SELECT * FROM parquet_home_sales LIMIT 5").show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [66]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
# SQL query to calculate the average price per view rating
query = """
SELECT
    view AS View_rating,
    CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS Average_Price
FROM
    parquet_home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view_rating DESC
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|View_rating|Average_Price|
+-----------+-------------+
|        100|$1,026,669.50|
|         99|$1,061,201.42|
|         98|$1,053,739.33|
|         97|$1,129,040.15|
|         96|$1,017,815.92|
|         95|$1,054,325.60|
|         94|$1,033,536.20|
|         93|$1,026,006.06|
|         92|  $970,402.55|
|         91|$1,137,372.73|
|         90|$1,062,654.16|
|         89|$1,107,839.15|
|         88|$1,031,719.35|
|         87|$1,072,285.20|
|         86|$1,070,444.25|
|         85|$1,056,336.74|
|         84|$1,117,233.13|
|         83|$1,033,965.93|
|         82|$1,063,498.00|
|         81|$1,053,472.79|
+-----------+-------------+
only showing top 20 rows

--- 0.5958065986633301 seconds ---


In [67]:
# Uncache the home_sales temporary table
spark.sql("UNCACHE TABLE home_sales")


DataFrame[]

In [68]:
# Check if the home_sales table is cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
if not is_cached:
    print("The 'home_sales' table is no longer cached.")
else:
    print("The 'home_sales' table is still cached.")




The 'home_sales' table is no longer cached.
